# Aula 7: Web Scrapping

Web scrapping ou raspagem de telas, aplicações e páginas são processos executados para coleta de dados das quais podem serem realizadas diretamente nas APIs que servem as aplicações, nas páginas estáticas ou simulando consições específicas do navegador.

Este tipo de coleta pode envolver grande complexidade se for adotado a coleta pelo navegador e diretamente pela página estática. Portanto procure identificar, com ferramentas de desenvolvedor, qual é a API utilizada e faça as chamadas diretamente delas.

Caso não seja possível a coleta direta das APIs, parta para a análise de página estática e, somente por último, analise a coleta pelo navegador

## Biblioteca Requests

A biblioteca requests permite acesso direto tanto a páginas estáticas quando a métodos das APIs Restful, sendo assim a primeira opção de coleta de dados dado sua simplicidade.

In [ ]:
import requests

Para exemplificar, vamos acessar a API do INPE para coleta de daods de metereologia. A API retorna os dados no formato XML.

In [ ]:
r = requests.get("http://servicos.cptec.inpe.br/XML/estacao/SBMT/condicoesAtuais.xml")

In [ ]:
r

<Response [200]>

In [ ]:
resultado = r.text
resultado

"<?xml version='1.0' encoding='ISO-8859-1'?><metar><codigo>SBMT</codigo><atualizacao>24/10/2021 12:00:00</atualizacao><pressao>1012</pressao><temperatura>23</temperatura><tempo>ps</tempo><tempo_desc>PredomÃ\xadnio de Sol</tempo_desc><umidade>61</umidade><vento_dir>260</vento_dir><vento_int>29</vento_int><visibilidade>>10000</visibilidade></metar>"

In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
xml = ET.fromstring(resultado)

for table in xml.getiterator('metar'):
    for child in table:
        print(child.tag, child.text)

codigo SBMT
atualizacao 24/10/2021 12:00:00
pressao 1012
temperatura 23
tempo ps
tempo_desc PredomÃ­nio de Sol
umidade 61
vento_dir 260
vento_int 29
visibilidade >10000


In [ ]:
r = requests.get("http://servicos.cptec.inpe.br/XML/cidade/241/dia/0/ondas.xml")
r

<Response [200]>

In [ ]:
resultado = r.text
resultado

"<?xml version='1.0' encoding='ISO-8859-1'?><cidade><nome>Rio de Janeiro</nome><uf>RJ</uf><atualizacao>24-10-2021</atualizacao><manha><dia>24-10-2021 12h Z</dia><agitacao>Fraco</agitacao><altura>0.9</altura><direcao>S</direcao><vento>3.9</vento><vento_dir>W</vento_dir></manha><tarde><dia>24-10-2021 18h Z</dia><agitacao>Fraco</agitacao><altura>0.7</altura><direcao>SSE</direcao><vento>4.1</vento><vento_dir>ENE</vento_dir></tarde><noite><dia>24-10-2021 21h Z</dia><agitacao>Fraco</agitacao><altura>0.7</altura><direcao>SSE</direcao><vento>7.8</vento><vento_dir>SSW</vento_dir></noite></cidade>"

In [ ]:
xml = ET.fromstring(resultado)

for table in xml.getiterator('cidade'):
    for child in table:
        print(child.tag, child.text)

nome Rio de Janeiro
uf RJ
atualizacao 24-10-2021
manha None
tarde None
noite None


## Parseando e simplificando dados de apresentação com a biblioteca Beautiful Soup

O Beautiful Soup nos traz maior facilidade de análise de páginas estáticas ao parsear o conteúdo HTML e permitir a busca de seus elementos.

Geralmente baixamos a página estática pelo Requests e analisamos o conteúdo com o Beaultiful Soup.

Vamos obter os dados de cotação de dólar a partir de uma faixa de datas, conforme abaixo.

Ao analisar a página de consulta, percebemos que ela envia uma requisição do tipo post para o mesmo endpoint da página estática, com isso extraímos os parâmetros e incluímos na requisição.

In [ ]:
parametros = {"RadOpcao": 1,"DATAINI": "25/09/2021", "DATAFIM": "24/10/2021", "ChkMoeda": 61}

In [ ]:
r = requests.post("https://ptax.bcb.gov.br/ptax_internet/consultaBoletim.do?method=consultarBoletim",params=parametros)
r

<Response [200]>

O resultado é uma outra página renderizada, da qual possui diversos componentes HTML. Nosso objetivo é extrair somente os dados de tabela e obter seus valores.

In [ ]:
resultado = r.text
resultado

'\r\n\r\n\r\n<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n<html>\r\n    <head>\r\n        <meta http-equiv="Content-Type" content="text/html; charset=ISO-8859-1">\r\n        <link rel="stylesheet" type="text/css" href="/ptax_internet/ncss/style.css">\r\n        <title></title>\r\n    </head>\r\n    <body>\r\n\r\n        <div style="text-align: center">\r\n            <br>\r\n            Cotações de Fechamento Ptax<sup>4/</sup> do DOLAR DOS EUA, Código da Moeda: 220, Símbolo da Moeda: USD, Tipo da Moeda: A, período de 25/09/2021 a 24/10/2021.\r\n            <br>\r\n            <br>\r\n\r\n            Clique para obter a tabela completa (\r\n            <a href="/ptax_internet/consultaBoletim.do?method=gerarCSVFechamentoMoedaNoPeriodo&ChkMoeda=61&DATAINI=25/09/2021&DATAFIM=24/10/2021">\r\n                <img src="/img/transferirA.GIF" border="0" longdesc="img">&nbsp; CSV - 2 KB\r\n            </a>)\r\n            <br>\r\n     

Nosso primeiro passo é parsear a página resultante com o Beautiful Soup.

In [ ]:
from bs4 import BeautifulSoup

Ao analisar a página resultante, notamos que a tabela possui 2 classes de estilo de interesse, responsável por variar a cor das linhas, e que nos ajudou a filtrar os componentes que precisams.

In [ ]:
bs = BeautifulSoup(resultado, 'html.parser')

items = bs.find_all('tr',{"class": ["fundoPadraoBClaro2","fundoPadraoBClaro3"]})
items

[<tr class="fundoPadraoBClaro2">
 <td>27/09/2021</td>
 <td>A</td>
 <td>5,3472</td>
 <td>5,3478</td>
 </tr>, <tr class="fundoPadraoBClaro3">
 <td>28/09/2021</td>
 <td>A</td>
 <td>5,4200</td>
 <td>5,4206</td>
 </tr>, <tr class="fundoPadraoBClaro2">
 <td>29/09/2021</td>
 <td>A</td>
 <td>5,4167</td>
 <td>5,4173</td>
 </tr>, <tr class="fundoPadraoBClaro3">
 <td>30/09/2021</td>
 <td>A</td>
 <td>5,4388</td>
 <td>5,4394</td>
 </tr>, <tr class="fundoPadraoBClaro2">
 <td>01/10/2021</td>
 <td>A</td>
 <td>5,3905</td>
 <td>5,3911</td>
 </tr>, <tr class="fundoPadraoBClaro3">
 <td>04/10/2021</td>
 <td>A</td>
 <td>5,4198</td>
 <td>5,4204</td>
 </tr>, <tr class="fundoPadraoBClaro2">
 <td>05/10/2021</td>
 <td>A</td>
 <td>5,4605</td>
 <td>5,4611</td>
 </tr>, <tr class="fundoPadraoBClaro3">
 <td>06/10/2021</td>
 <td>A</td>
 <td>5,5091</td>
 <td>5,5097</td>
 </tr>, <tr class="fundoPadraoBClaro2">
 <td>07/10/2021</td>
 <td>A</td>
 <td>5,5134</td>
 <td>5,5140</td>
 </tr>, <tr class="fundoPadraoBClaro3">
 <td

In [ ]:
items[0]

<tr class="fundoPadraoBClaro2">
<td>27/09/2021</td>
<td>A</td>
<td>5,3472</td>
<td>5,3478</td>
</tr>

In [ ]:
items[1]

<tr class="fundoPadraoBClaro3">
<td>28/09/2021</td>
<td>A</td>
<td>5,4200</td>
<td>5,4206</td>
</tr>

Depois de analisar cada linha, analisaremos cada coluna e assim compor nosso dataframe com a evolução da cotação do dólar.

In [ ]:
sub_item = items[0].findChildren('td')
sub_item

[<td>27/09/2021</td>, <td>A</td>, <td>5,3472</td>, <td>5,3478</td>]

A tabela tem uma ordem para cada coluna, sendo a data, o tipo, o valor de compra e venda.

In [ ]:
for i in items:
  sub_items = i.findChildren("td")
  for si in sub_items:
    print(si.text)
  break

27/09/2021
A
5,3472
5,3478


In [ ]:
data, tipo, compra, venda = [], [], [], []

In [ ]:
campo = "data"
for i in items:
  sub_items = i.findChildren("td")
  for si in sub_items:
    if campo == "data":
      data.append(si.text)
      campo = "tipo"
    elif campo == "tipo":
      campo = "compra"
      tipo.append(si.text)
    elif campo == "compra":
      campo = "venda"
      compra.append(si.text)
    else:
      campo = "data"
      venda.append(si.text)

In [ ]:
data

['27/09/2021',
 '28/09/2021',
 '29/09/2021',
 '30/09/2021',
 '01/10/2021',
 '04/10/2021',
 '05/10/2021',
 '06/10/2021',
 '07/10/2021',
 '08/10/2021',
 '11/10/2021',
 '13/10/2021',
 '14/10/2021',
 '15/10/2021',
 '18/10/2021',
 '19/10/2021',
 '20/10/2021',
 '21/10/2021',
 '22/10/2021']

In [ ]:
compra

['5,3472',
 '5,4200',
 '5,4167',
 '5,4388',
 '5,3905',
 '5,4198',
 '5,4605',
 '5,5091',
 '5,5134',
 '5,5078',
 '5,5155',
 '5,5464',
 '5,4982',
 '5,4504',
 '5,5187',
 '5,5515',
 '5,5565',
 '5,6417',
 '5,7111']

In [ ]:
import pandas as pd

In [ ]:


df = pd.DataFrame({"Data": data, "Tipo": tipo, "Compra": compra, "Venda": venda})
df

,Data,Tipo,Compra,Venda
0,27/09/2021,A,"5,3472","5,3478"
1,28/09/2021,A,"5,4200","5,4206"
2,29/09/2021,A,"5,4167","5,4173"
3,30/09/2021,A,"5,4388","5,4394"
4,01/10/2021,A,"5,3905","5,3911"
5,04/10/2021,A,"5,4198","5,4204"
6,05/10/2021,A,"5,4605","5,4611"
7,06/10/2021,A,"5,5091","5,5097"
8,07/10/2021,A,"5,5134","5,5140"
9,08/10/2021,A,"5,5078","5,5084"


## Utilizando Selenium para coleta de dados

O Selenium é uma biblioteca disponível em várias linguagens de programação cujo o principal objetivo foi de automatizar testes de interface. Utilizando essa capacidade de interação também podemos utilizar para coletar dados de sites onde a manipulação do navegador torna o processo ou obrigatório ou mais fácil, com o filtro por XPath, ausente no Beautiful Soup.

A instalação do Colab requer os passos a seguir:

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 954 kB 5.3 MB/s 
     |████████████████████████████████| 138 kB 55.8 MB/s 
     |████████████████████████████████| 356 kB 59.2 MB/s 
     |████████████████████████████████| 55 kB 3.2 MB/s 
     |████████████████████████████████| 3.5 MB 44.1 MB/s 
     |████████████████████████████████| 54 kB 2.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.down

Somente no Colab, precisamos configurar de forma especial o driver (navegador) utilizado.

In [ ]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
from selenium.webdriver.common.by import By
import time 

Esta função coleta os dados de um determinado produto no e-commerce Kabum.

In [ ]:
def scraping_pages():

    url_root = 'https://www.kabum.com.br/produto/109633/teclado-sem-fio-logitech-mx-keys-bluetooth-tecnologia-flow-us-cinza-920-009297'

    driver.get(url_root)
    time.sleep(10)
      
    produto = driver.find_element(By.XPATH,'//*[@id="__next"]/main/article/section/div[2]/h1')
    preco = driver.find_element(By.XPATH,'//*[@id="blocoValores"]/div[2]/div/h4')             
    avaliacao = driver.find_element(By.XPATH,'//*[@id="__next"]/main/article/section/div[2]/div/div[1]/div[1]/div[3]/div[2]')
    
    print(produto.text)
    print(preco.text)
    print(avaliacao.text)


In [ ]:
scraping_pages()

Teclado Sem Fio Logitech MX Keys, Bluetooth, Tecnologia Flow, US, Cinza - 920-009297
R$ 649,90
(71)


Podemos explorar a paginação dos anúncios em busca de mais dados.

In [ ]:
def get_anuncios_kabum(link):
  url_root = link
  driver.get(url_root)
  time.sleep(5)
  
  existing_items = False

  produtos_div = driver.find_element(By.XPATH,'//*[@id="listing"]/article/section/div[2]/div/main')
  lista_items = produtos_div.find_elements(By.XPATH, '//div/h2')
  
  for item in lista_items:
    print(item.text)

    if item.text != '':
      existing_items = True
  
  return existing_items

Teste de coleta de uma página individual.

In [ ]:
get_anuncios_kabum('https://www.kabum.com.br/promocao/MENU_ELETRONICOS?page_number=2&page_size=20&facet_filters=&sort=most_searched')

Vídeo Porteiro KaBuM! Smart, Wi-Fi, Campainha, Áudio Bidirecional, 1080p, Alexa, Google Assistant
<selenium.webdriver.remote.webelement.WebElement (session="f90a86ffd85642720c13ff3af2caeaca", element="09b27112-ad9f-4e37-bd41-a786fd024d1f")>
Soundbar LG 440W RMS, 3.1.2 Ch, Google Assistente, DTs X, Dolby Atmos, Bluetooth - SN8YG
<selenium.webdriver.remote.webelement.WebElement (session="f90a86ffd85642720c13ff3af2caeaca", element="216c2341-5628-4cfb-82a7-2c0a47e7b9d5")>
Roteador TP-Link Wireless (Sistema Mesh) AC1300 1300Mbps - DECO M5
<selenium.webdriver.remote.webelement.WebElement (session="f90a86ffd85642720c13ff3af2caeaca", element="44236dbe-a3f3-42ce-aaf7-edde32692865")>
Smart TV LG 75 4K UHD 75UP8050, com WiFi e Bluetooth, HDR, Inteligência Artificial, ThinQ Smart Magic, Google Alexa - 75UP8050PSB
<selenium.webdriver.remote.webelement.WebElement (session="f90a86ffd85642720c13ff3af2caeaca", element="0aebc59f-fd07-4868-8e51-870b81da604b")>
Smartwatch Samsung Galaxy Watch 3 45mm LTE, 

True

Incluindo o algortimo para busca e paginação automática.

In [ ]:
page_nr = 0
items = True

while items:
  page_nr += 1
  url = "https://www.kabum.com.br/promocao/MENU_ELETRONICOS?page_number="+str(page_nr) + "&page_size=20&facet_filters=&sort=most_searched"
  print("Página " + str(page_nr))
  print(url)
  items = get_anuncios_kabum(url)

Página 1
https://www.kabum.com.br/promocao/MENU_ELETRONICOS?page_number=1&page_size=20&facet_filters=&sort=most_searched
Smartwatch Samsung Galaxy Watch Active 2, 44mm, Wi-Fi, Touchscreen, Monitor Cardíaco, Preto - SM-R820NZKPZTO
Tablet Samsung Galaxy Tab A7, Wi-Fi, Android 10, 64GB, 8MP, Tela 10.4´, Grafite - SM-T500NZAQZTO
Fone de Ouvido Bluetooth Samsung Galaxy Buds Live, com Microfone, Recarregável, Preto - SM-R180NZKPZTO
Smart TV Samsung 50´ 4K QLED 50Q60A, Tela Infinita, Processador IA, HDR10+, Design Slim, Alexa Built In - QN50Q60AAGXZD
Roteador Wireless D-Link MU-MIMO Gigabit AC1200, 1200Mbps, 6 Antenas - DIR-846
Echo Dot (4ª Geração) com Alexa, Amazon Smart Speaker Azul - B084KV8YRR
AirPods com Estojo de Recarga, Branco - MV7N2AM/A
Fone de Ouvido Samsung Galaxy Buds Pro, Cancelamento de Ruído, Preto - SM-R190NZKPZTO
Smart TV TCL 55" P715 LED 4K UHD, WiFi, Bluetooth, 3x HDMI, 2x USB, HDR, Google Assistant, Android TV, Borda Ultrafina - 55P715
Samsung Smart TV 70´´ UHD 4K 70AU77